In [1]:
# Import Data
import pandas as pd
import arcpy
import os
from datetime import datetime
import pathlib
import numpy as np
from arcgis import GIS
from arcgis.features import FeatureLayer
from arcgis.geometry import SpatialReference
from functools import reduce
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

tycdatafile_path = r"F:\Research and Analysis\Vegetation\TYC\TYC Appendix C 2023.xlsx"
sheet_name = '2023'  # Name of the sheet to read

df = pd.read_excel(tycdatafile_path, sheet_name=sheet_name)#, header=[0, 1])

#drop row 0
df = df.drop(0)

#make a copy of this data frame that includes Site Name and Columns with years
df2 = df.copy()

#drop all columns that are not Site Name or a year
df2 = df2[[' SITE NAME', 2023, 2022, 2021, 2020]]

#drop rows 65-71
df2 = df2.drop([64,65,66,67,68,69,70,71])

df_long = df2.melt(id_vars=[' SITE NAME'], var_name='Year', value_name='Value')






In [2]:

# set environement workspace to in memory 
arcpy.env.workspace = 'memory'
# # clear memory workspace
# arcpy.management.Delete('memory')

# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)

# function to get sql connection for tabular TRPA data Collection.sde?
# db options are 'tabular' or 'tahoebmpsde'??
def get_conn(db):
    # Get database user and password from environment variables on machine running script
    db_user             = os.environ.get('DB_USER')
    db_password         = os.environ.get('DB_PASSWORD')

    # driver is the ODBC driver for SQL Server
    driver              = 'ODBC Driver 17 for SQL Server'
    # server names are
    sql_12              = 'sql12'
    
    # make it case insensitive
    db = db.lower()
    # make sql database connection with pyodbc
    if db == 'sde':
        connection_string = f"DRIVER={driver};SERVER={sql_12};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    # else return None
    else:
        engine = None
    # connection file to use in pd.read_sql
    return engine


# network path to connection files
filePath = "F:\\GIS\\DB_CONNECT"
### SDE Vector is where the data will go into staging tables 
sdeVector    = os.path.join(filePath, "Vector.sde")
# local variables sdata is starting data and f data is finishing datatables
sdemonitoring= os.path.join(sdeVector, "sde.SDE.Monitoring")
#make this a spatial df
tycdata_path = os.path.join(sdemonitoring, "sde.SDE.Yellow_Cress")

# Initialize an empty list to store the data
tycdata = []

# Extract site names and coordinates
with arcpy.da.SearchCursor(tycdata_path, ["SITE_NAME", "SHAPE@XY", "OWNER", "LTINFO_HYPERLINK", "SHAPE"]) as cursor:
    for row in cursor:
        site_name = row[0]  # Site name
        longitude, latitude = row[1]  # Extract X (longitude) and Y (latitude)
        owner = row[2]  # Owner
        ltinfo_hyperlink = row[3]  # LTINFO Hyperlink
        tycdata.append([site_name, latitude, longitude, owner, ltinfo_hyperlink])

# Create a pandas DataFrame
tyc_df = pd.DataFrame(tycdata, columns=["SITE_NAME", "Latitude", "Longitude", "OWNER", "LTINFO_HYPERLINK"])


In [3]:

# Standardize column name in df before processing
df_long.rename(columns={' SITE NAME': 'SITE_NAME'}, inplace=True)

# # Strip whitespace and standardize case to avoid mismatches
df_long['SITE_NAME'] = df_long['SITE_NAME'].astype(str).str.strip().str.lower()
tyc_df['SITE_NAME'] = tyc_df['SITE_NAME'].str.strip().str.lower()
#keep only site name, owner, 
tyc_df

# # Merge the two DataFrames on 'Site_Name'
merged_df = pd.merge(df_long, tyc_df, on='SITE_NAME', how='left')
# # Display the merged DataFrame
#print(merged_df.head())



In [ ]:
print(df.columns)




In [4]:
type_mapping = {
    #'int64': 'LONG',
    'float64': 'DOUBLE',
    'object': 'TEXT',
    'string': 'TEXT',
    #'datetime64[ns]': 'DATE'
}
 
# Set up geodatabase and output table name
gdb_path = r"F:\Research and Analysis\Workspace\Sarah\Scratch.gdb"
output_table = "tyc_temp"
output_path = f"{gdb_path}\\{output_table}"

#Delete existing table if it exists
if arcpy.Exists(output_path):
    arcpy.management.Delete(output_path)
    print(f"Deleted existing table: {output_table}")

# Create the table in the geodatabase